求导是几乎所有深度学习优化算法的关键步骤。 虽然求导的计算很简单，只需要一些基本的微积分。
但对于复杂的模型，手工进行更新是一件很痛苦的事情（而且经常容易出错）。

深度学习框架通过自动计算导数，即自动微分（automatic differentiation）来加快求导。
实际中，根据我们设计的模型，系统会构建一个计算图（computational graph），
来跟踪计算是哪些数据通过哪些操作组合起来产生输出。
自动微分使系统能够随后反向传播梯度。
这里，反向传播（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

### 3.5.1 一个简答的例子
作为一个演示例子，假设我们想对函数 y=2x⊤x 关于列向量 x 求导。 首先，我们创建变量x并为其分配一个初始值。

In [42]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [43]:
# 在我们计算 y 关于 x 的梯度之前，我们需要一个地方来存储梯度。 重要的是，我们不会在每次对一个参数求导时都分配新的内存。 因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。 注意，一个标量函数关于向量 x 的梯度是向量，并且与 x 具有相同的形状。
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

In [44]:
y = 2 * torch.dot(x,x)# 2 * (1*1 + 4*4 + 9*9)
y

tensor(28., grad_fn=<MulBackward0>)

x是一个长度为4的向量，计算x和x的点积，得到了我们赋值给y的标量输出。
接下来，我们通过调用反向传播函数来自动计算y关于x每个分量的梯度，并打印这些梯度。

In [45]:
y.backward()
x.grad # y反向传播，修改了x的梯度

tensor([ 0.,  4.,  8., 12.])

In [46]:
# 函数 y=2x⊤x 关于 x 的梯度应为 4x 。 让我们快速验证这个梯度是否计算正确。
x.grad == 4 * x

tensor([True, True, True, True])

In [47]:
# 现在让我们计算x的另一个函数。
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x

tensor([0., 1., 2., 3.], requires_grad=True)

In [52]:
x.grad.zero_()
y = x.sum()
y.backward()# 新的y反向传播，修改了x的梯度，导致梯度也发生变化，新的梯度为1
x.grad,y

(tensor([1., 1., 1., 1.]), tensor(6., grad_fn=<SumBackward0>))

### 2.5.2 非标量变量的反向传播
当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵。
对于高阶和高维的y和x，求导的结果可以是一个高阶张量。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括深度学习中），
但当我们调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
这里，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。

In [60]:
x.grad.zero_()
y = x * x
f'x: {x}, y: {y}, y.sum: {y.sum()}'

'x: tensor([0., 1., 2., 3.], requires_grad=True), y: tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>), y.sum: 14.0'

In [61]:
# 等价于y.backward(torch.ones(len(x)))
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
y.sum().backward()# y' = 2x
x.grad

tensor([0., 2., 4., 6.])

In [65]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u *  x
z.sum().backward() # 计算z=u*x关于x的偏导数，同时将u作为常数处理， 而不是z=x*x*x关于x的偏导数
x.grad == u, x.grad, u.grad

(tensor([True, True, True, True]), tensor([0., 1., 4., 9.]), None)

由于记录了y的计算结果，我们可以随后在y上调用反向传播， 得到y=x\*x关于的x的导数，即2\*x。

In [51]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

### 2.5.4 Python控制流的梯度计算
使用自动微分的一个好处是：
即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），
我们仍然可以计算得到的变量的梯度。 在下面的代码中，while循环的迭代次数和if语句的结果都取决于输入a的值。

In [67]:
def f(a):
    b = 2 * a
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [74]:
a = torch.randn(size=(),requires_grad=True)
d = f(a)
d.backward()
d,a

(tensor(1611.5497, grad_fn=<MulBackward0>), tensor(0.0984, requires_grad=True))

我们现在可以分析上面定义的f函数。
请注意，它在其输入a中是分段线性的。
换言之，对于任何a，存在某个常量标量k，使得f(a)=k*a，
其中k的值取决于输入a。
因此，我们可以用d/a验证梯度是否正确。

In [73]:
a.grad == d / a

tensor(True)